# Example usage with the CLI
This notebook demonstrates usage of `petab_select` to perform model selection with commands.

Note that the criterion values in this notebook are for demonstrative purposes only, and are not real (the models were not calibrated).

In [1]:
# Cleanup the state and candidate models output by a previous run of this notebook
import os

try:
    os.remove('output/state.dill')
except FileNotFoundError:
    pass

try:
    os.remove('output/models.yaml')
except FileNotFoundError:
    pass

## First iteration

In each call to `petab_select search`, the following options are required:
- `-y`: The PEtab Select problem YAML file;
- `-s`: A file that is used to stored the state of the problem (e.g., such that models are not repeated);
- `-o`: A file to store the output of the model space search; and
- `-m`: The method used to search or identify models in the model space.

Other options can be viewed with `petab_select search --help`.

In this initial call, a PEtab Select problem is used to identify possible models for selection. The brute force method is used, which normally outputs all possible models. Here, the number of models in the output is explicitly limited to `3`. Subsequent calls with the same command will output different models.

In [2]:
%%bash

petab_select candidates \
-y model_selection/petab_select_problem.yaml \
-s output/state.dill \
-o output/models_1.yaml \
-m brute_force \
--relative-paths \
-l 3

/home/dilan/Documents/future_annex/model_selection/packages/petab_select/petab_select/model_subspace.py:236: UserWarning: The candidate space has already reached its limit of accepted models.
  warnings.warn('The candidate space has already reached its limit of accepted models.')  # noqa: E501


The output format is a list of the PEtab Select model YAML format.

In [3]:
with open('output/models_1.yaml', 'r') as f:
    print(f.read())

- criteria: {}
  estimated_parameters: {}
  model_hash: 65f94e223024ef684fe3e1a3aa2a54cc3ffd08895fbe4539512522d49d703ceda483aff4aa207b160dc358c458b76b25d88fbd94cacfc78bd0c70f4a46a42191
  model_id: 65f94e223024ef684fe3e1a3aa2a54cc3ffd08895fbe4539512522d49d703ceda483aff4aa207b160dc358c458b76b25d88fbd94cacfc78bd0c70f4a46a42191
  model_subspace_id: M1_0
  model_subspace_indices:
  - 0
  - 0
  - 0
  parameters:
    k1: 0
    k2: 0
    k3: 0
  petab_yaml: ../model_selection/petab_problem.yaml
  predecessor_model_hash: null
- criteria: {}
  estimated_parameters: {}
  model_hash: 112c344171a01874a0b400640c2e0f72f2924b91712966cb868bf53b6d8ce2d09bb8e56f52b5aaca506a64754629147047646ea0c0cf568d76e74df2c5e2487a
  model_id: 112c344171a01874a0b400640c2e0f72f2924b91712966cb868bf53b6d8ce2d09bb8e56f52b5aaca506a64754629147047646ea0c0cf568d76e74df2c5e2487a
  model_subspace_id: M1_1
  model_subspace_indices:
  - 0
  - 0
  - 0
  parameters:
    k1: 0.2
    k2: 0.1
    k3: estimate
  petab_yaml: ../model_sel

## Second iteration

Between iterations, the models from the first iteration have been calibrated, and the model with the best criterion value is `M1_2_0`. Here, PEtab Select will identify `M1_2_0` as the best model from a YAML file of calibrated models. In the following iterations, a specific model will be provided.

In [4]:
with open('model_selection/calibrated/calibrated_first_iteration.yaml', 'r') as f:
    print(f.read())

- criteria:
    AIC: 180
  model_id: M1_0_0
  parameters:
    k1: 0.0
    k2: 0.0
    k3: 0.0
  estimated_parameters: null
  petab_yaml: ../petab_problem.yaml
- criteria:
    AIC: 100
  model_id: M1_1_0
  parameters:
    k1: 0.2
    k2: 0.1
    k3: estimate
  estimated_parameters:
    k3: 0.0
  petab_yaml: ../petab_problem.yaml
- criteria:
    AIC: 50
  model_id: M1_2_0
  parameters:
    k1: 0.2
    k2: estimate
    k3: 0.0
  estimated_parameters:
    k2: 0.05
  petab_yaml: ../petab_problem.yaml





This is used as the predecessor model for the next search, this time using the forward method. The same state file can be used in each call. If the output file already exists, it will be overwritten.

In [5]:
%%bash

petab_select candidates \
-y model_selection/petab_select_problem.yaml \
-s output/state.dill \
-o output/models_2.yaml \
-m forward \
-b model_selection/calibrated/calibrated_first_iteration.yaml \
--relative-paths

`M1_2_0` has one estimated parameter, `k2` [1]. As expected, the new candidates identified with the forward method have two estimated parameters, and one of them is `k2`.

[1] There may be additional estimated parameters specified in the PEtab problem.

In [6]:
with open('output/models_2.yaml', 'r') as f:
    print(f.read())

- criteria: {}
  estimated_parameters: {}
  model_hash: 38c95dd428b3e31da6969a50db4a1ccbcefe6d8824617d27ec2360e57d55647a25f3fdd45e5f0270786698606cbe496cd94be9495986dade4d1f1d166a4bf911
  model_id: 38c95dd428b3e31da6969a50db4a1ccbcefe6d8824617d27ec2360e57d55647a25f3fdd45e5f0270786698606cbe496cd94be9495986dade4d1f1d166a4bf911
  model_subspace_id: M1_4
  model_subspace_indices:
  - 0
  - 0
  - 0
  parameters:
    k1: 0.2
    k2: estimate
    k3: estimate
  petab_yaml: ../model_selection/petab_problem.yaml
  predecessor_model_hash: b91dd6d1144253c4462597de1fff07cb20c6e45ab33d55a8282bf2b3df25d1226ad19baf488a96852a626df1b56e1e618929f0c0627ed6f2c3f69ebb0bcbb6fc
- criteria: {}
  estimated_parameters: {}
  model_hash: db8700c079c8347123adc89b7f5112256c4aaebd2af0f6e32e7582f398b2c1e5e85e588cdcc56bab054c001b96a9b42b02174266927f879d7f78e8ac5d2c33e6
  model_id: db8700c079c8347123adc89b7f5112256c4aaebd2af0f6e32e7582f398b2c1e5e85e588cdcc56bab054c001b96a9b42b02174266927f879d7f78e8ac5d2c33e6
  model_sub

## Third iteration

Models `M1_4_0` and `M1_6_0` were calibrated, with `M1_4_0` having the superior criterion value, so is used as the predecessor model in the next forward search.

In [7]:
with open('model_selection/calibrated/M1_4_0.yaml', 'r') as f:
    print(f.read())

criteria:
  AIC: 30
model_id: M1_4_0
parameters:
  k1: 0.2
  k2: estimate
  k3: estimate
estimated_parameters:
  k2: 0.15
  k3: 0.0
petab_yaml: ../petab_problem.yaml



In [8]:
%%bash

petab_select candidates \
-y model_selection/petab_select_problem.yaml \
-s output/state.dill \
-o output/models_3.yaml \
-m forward \
-p model_selection/calibrated/M1_4_0.yaml \
--relative-paths

The model space contains only one model with 3 or more estimated parameters, which is a valid candidate model in this case.

In [9]:
with open('output/models_3.yaml', 'r') as f:
    print(f.read())

- criteria: {}
  estimated_parameters: {}
  model_hash: 7c105406ec11716473939a0bbb5281066c1014b54e2480ba126030f5c18a597a27a2ca9247aa60d8262f488165079d1c9e040f9d712ec4e19c2d2122a586f3e5
  model_id: 7c105406ec11716473939a0bbb5281066c1014b54e2480ba126030f5c18a597a27a2ca9247aa60d8262f488165079d1c9e040f9d712ec4e19c2d2122a586f3e5
  model_subspace_id: M1_7
  model_subspace_indices:
  - 0
  - 0
  - 0
  parameters:
    k1: estimate
    k2: estimate
    k3: estimate
  petab_yaml: ../model_selection/petab_problem.yaml
  predecessor_model_hash: 3a5dadef211388a3da6610b1c057d16a135dcddd5273efa88b1875bbafb153afeb4398a8ed02cf04e0fa7c04348265bebe2b02b8c0def80e2fb6f329e469e7f7



## Fourth iteration
As there are no models in the model space with additional estimated parameters, subsequent forward searches will return no candidate models.

In [10]:
with open('model_selection/calibrated/M1_7_0.yaml', 'r') as f:
    print(f.read())

criteria:
  AIC: 20
model_id: M1_7_0
parameters:
  k1: estimate
  k2: estimate
  k3: estimate
estimated_parameters:
  k1: 0.25
  k2: 0.1
  k3: 0.0
petab_yaml: ../petab_problem.yaml



In [11]:
%%bash

petab_select candidates \
-y model_selection/petab_select_problem.yaml \
-s output/state.dill \
-o output/models_4.yaml \
-m forward \
-p model_selection/calibrated/M1_7_0.yaml \
--relative-paths

In [12]:
with open('output/models_4.yaml', 'r') as f:
    print(f.read())

null
...



## Fifth iteration
Although no additional models are found with a forward search initialized at the best model so far (`M1_7_0`), there are additional models in the model space that are yet to be calibrated, which can be identified with the brute force method.

In [13]:
%%bash

petab_select candidates \
-y model_selection/petab_select_problem.yaml \
-s output/state.dill \
-o output/models.yaml \
-m brute_force \
-e output/models_1.yaml \
-e output/models_2.yaml \
-e output/models_3.yaml \
-e output/models_4.yaml \
--relative-paths

In [14]:
with open('output/models.yaml', 'r') as f:
    print(f.read())

- criteria: {}
  estimated_parameters: {}
  model_hash: b7584bfd6f35206dfe32fa0143e53cea808faf965e0c0547bf6ee1cdce7a75cd3ff0aa2bcb1faa27625166454f83e3fcac52cdf43b28e8186fff9a01ac3f8006
  model_id: b7584bfd6f35206dfe32fa0143e53cea808faf965e0c0547bf6ee1cdce7a75cd3ff0aa2bcb1faa27625166454f83e3fcac52cdf43b28e8186fff9a01ac3f8006
  model_subspace_id: M1_3
  model_subspace_indices:
  - 0
  - 0
  - 0
  parameters:
    k1: estimate
    k2: 0.1
    k3: 0
  petab_yaml: ../model_selection/petab_problem.yaml
  predecessor_model_hash: null
- criteria: {}
  estimated_parameters: {}
  model_hash: de4a2f17d8b0228a31d7451631cf3662d0ecf4dc7738ab6ca3d1de65e817844c9c1df806ec9daf81644b9c10f00185dc8c8de880d9db23a98acadb817f5d481c
  model_id: de4a2f17d8b0228a31d7451631cf3662d0ecf4dc7738ab6ca3d1de65e817844c9c1df806ec9daf81644b9c10f00185dc8c8de880d9db23a98acadb817f5d481c
  model_subspace_id: M1_5
  model_subspace_indices:
  - 0
  - 0
  - 0
  parameters:
    k1: estimate
    k2: 0.1
    k3: estimate
  petab_yaml

# Post-processing
After the selection algorithm has terminated, the best model can be stored separately by supplying a list of calibrated models.

In [15]:
%%bash

petab_select best \
-y model_selection/petab_select_problem.yaml \
-m model_selection/calibrated/calibrated_first_iteration.yaml \
-o output/best_model.yaml \
-s output/state.dill \
--relative-paths

In [16]:
with open('output/best_model.yaml', 'r') as f:
    print(f.read())

criteria:
  AIC: 50
estimated_parameters:
  k2: 0.05
model_hash: null
model_id: M1_2_0
model_subspace_id: null
model_subspace_indices: null
parameters:
  k1: 0.2
  k2: estimate
  k3: 0.0
petab_yaml: ../model_selection/petab_problem.yaml
predecessor_model_hash: null



This model can be converted to a PEtab problem with either `model_to_petab` or `models_to_petab`.

In [17]:
%%bash

petab_select model_to_petab \
-y output/best_model.yaml \
-o output/best_model_petab

output/best_model_petab/problem.yaml
